# Base Case

In [152]:
# Feb 23
from fastai.tabular import *
from fastai.metrics import fbeta
from tqdm import tqdm

In [153]:
full_df = pd.read_csv('../data/datablob_added_facies.csv', index_col=0)
full_df = full_df[pd.notnull(full_df['fm'])]

In [189]:
df = pd.read_csv('../data/with_lith.csv', index_col=0)

# Load data + Preprocess

In [190]:
df.columns

Index(['tvdss', 'BS', 'CALI', 'DENS', 'DRHO', 'DTC', 'GR', 'NEUT', 'PEF',
       'RESD', 'RESM', 'RESS', 'SP', 'fm', 'well', 'DTS', 'GR_CORR',
       'NEUT_CORR', 'RESD_CORR', 'RESS_CORR', 'TEMP', 'TENS', 'facies',
       'lith'],
      dtype='object')

In [191]:
df.head()

,tvdss,BS,CALI,DENS,DRHO,DTC,GR,NEUT,PEF,RESD,...,well,DTS,GR_CORR,NEUT_CORR,RESD_CORR,RESS_CORR,TEMP,TENS,facies,lith
1717,75.214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO FACIES,Sand
1718,75.214,8.5,9.767,2.137,-0.007,131.311,84.724,0.429,2.262,78.705,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand
1719,75.366,8.5,9.905,2.104,-0.005,131.458,83.405,0.408,2.396,75.329,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand
1720,75.518,8.5,9.958,2.136,-0.003,131.971,83.974,0.388,2.399,80.198,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand
1721,75.671,8.5,9.951,2.117,0.000,131.583,88.314,0.421,2.278,83.477,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand


In [192]:
labels = df.lith.unique()

In [193]:
dep_var = 'lith'
cat_names = []
cont_names = ['DENS', 'DTS', 'GR', 'PEF', 'RESD', 'RESM', 'RESS', 'NEUT', 'SP']
procs = [FillMissing, Categorify, Normalize]

In [194]:
test = TabularList.from_df(df, path='.', cat_names=cat_names, cont_names=cont_names)

In [195]:
data = (TabularList.from_df(df, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .random_split_by_pct()
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [196]:
learn = tabular_learner(data, layers=[200,100], metrics=[accuracy])

In [197]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.066971,0.039139,0.980647


In [200]:
preds,y = learn.get_preds(ds_type=DatasetType.Test)

In [201]:
len(preds)

242088

In [202]:
type(np.array(preds))

numpy.ndarray

In [203]:
preds[0:5]

tensor([[9.9984e-01, 1.5902e-04],
        [1.0000e+00, 2.2314e-08],
        [1.0000e+00, 2.2328e-08],
        [1.0000e+00, 2.2460e-08],
        [1.0000e+00, 2.2065e-08]])

In [131]:
y = list(map(lambda x:labels[x.item()],y))

In [204]:
result = list(map(lambda x:labels[np.argmax(x).item()],preds))

In [185]:
result[0:5]

['Sand', 'Sand', 'Sand', 'Sand', 'Sand']

In [205]:
lala = pd.Series(result)

In [206]:
len(lala)

242088

In [207]:
len(full_df)

242088

In [208]:
full_df['lith_pred'] = pd.Series(result).T.tolist()

In [209]:
df.head()

,tvdss,BS,CALI,DENS,DRHO,DTC,GR,NEUT,PEF,RESD,...,well,DTS,GR_CORR,NEUT_CORR,RESD_CORR,RESS_CORR,TEMP,TENS,facies,lith
1717,75.214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO FACIES,Sand
1718,75.214,8.5,9.767,2.137,-0.007,131.311,84.724,0.429,2.262,78.705,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand
1719,75.366,8.5,9.905,2.104,-0.005,131.458,83.405,0.408,2.396,75.329,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand
1720,75.518,8.5,9.958,2.136,-0.003,131.971,83.974,0.388,2.399,80.198,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand
1721,75.671,8.5,9.951,2.117,0.000,131.583,88.314,0.421,2.278,83.477,...,cheal-a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SH,Sand


In [210]:
df.to_csv('../data/lith_pred.csv')

In [81]:
result[0:5]

['Sand', 'Sand', 'Sand', 'Sand', 'Sand']

In [25]:
preds = []
for i in tqdm(range(len(df))):
#     print(i,)
    row = df.iloc[i]
    p = learn.predict(row)
#     print(p[0])
    preds.append(str(p[0]))

  0%|          | 556/242088 [00:15<2:00:55, 33.29it/s]

KeyboardInterrupt: 

  0%|          | 556/242088 [00:30<2:00:55, 33.29it/s]